## Setup

In [20]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
import os

DEBUG_MODE = False
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"
try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

if IN_COLAB or IN_GITHUB:
    %pip install transformer_lens
    %pip install torchtyping
    # Install my janky personal plotting utils
    %pip install git+https://github.com/neelnanda-io/neel-plotly.git
    # Install another version of node that makes PySvelte work way faster
    %pip install circuitsvis
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3

Running as a Jupyter notebook - intended for development only!
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\Danik\AppData\Local\Temp\ipykernel_19484\3902491722.py:18: DeprecationWarning:

`magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).

C:\Users\Danik\AppData\Local\Temp\ipykernel_19484\3902491722.py:19: DeprecationWarning:

`magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).



In [21]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"

In [22]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional, Callable
from functools import partial
import copy
import itertools
import json

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML, Markdown

In [23]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import (
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)

In [24]:
from neel_plotly import line, imshow, scatter
import plotly.io as pio
pio.renderers.default = 'vscode'

import transformer_lens.patching as patching

from typing_extensions import Literal

## IOI patching setup

In [25]:
model = HookedTransformer.from_pretrained("gpt2-small")


model.set_use_attn_result(True)
model.set_use_attn_in(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


In [26]:
prompts = [
    "When John and Mary went to the shops, John gave the bag to",
    "When John and Mary went to the shops, Mary gave the bag to",
    "When Tom and James went to the park, James gave the ball to",
    "When Tom and James went to the park, Tom gave the ball to",
    "When Dan and Sid went to the shops, Sid gave an apple to",
    "When Dan and Sid went to the shops, Dan gave an apple to",
    "After Martin and Amy went to the park, Amy gave a drink to",
    "After Martin and Amy went to the park, Martin gave a drink to",
]
answers = [
    (" Mary", " John"),
    (" John", " Mary"),
    (" Tom", " James"),
    (" James", " Tom"),
    (" Dan", " Sid"),
    (" Sid", " Dan"),
    (" Martin", " Amy"),
    (" Amy", " Martin"),
]

clean_tokens = model.to_tokens(prompts)
# Swap each adjacent pair, with a hacky list comprehension
corrupted_tokens = clean_tokens[
    [(i + 1 if i % 2 == 0 else i - 1) for i in range(len(clean_tokens))]
]
print("Clean string 0", model.to_string(clean_tokens[0]))
print("Corrupted string 0", model.to_string(corrupted_tokens[0]))

answer_token_indices = torch.tensor(
    [
        [model.to_single_token(answers[i][j]) for j in range(2)]
        for i in range(len(answers))
    ],
    device=model.cfg.device,
)
print("Answer token indices", answer_token_indices)

Clean string 0 <|endoftext|>When John and Mary went to the shops, John gave the bag to
Corrupted string 0 <|endoftext|>When John and Mary went to the shops, Mary gave the bag to
Answer token indices tensor([[ 5335,  1757],
        [ 1757,  5335],
        [ 4186,  3700],
        [ 3700,  4186],
        [ 6035, 15686],
        [15686,  6035],
        [ 5780, 14235],
        [14235,  5780]])


In [27]:
def get_logit_diff(logits, answer_token_indices=answer_token_indices):
    if len(logits.shape) == 3:
        # Get final logits only
        logits = logits[:, -1, :]
    correct_logits = logits.gather(1, answer_token_indices[:, 0].unsqueeze(1))
    incorrect_logits = logits.gather(1, answer_token_indices[:, 1].unsqueeze(1))
    return (correct_logits - incorrect_logits).mean()


clean_logits, clean_cache = model.run_with_cache(clean_tokens)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)

clean_logit_diff = get_logit_diff(clean_logits, answer_token_indices).item()
print(f"Clean logit diff: {clean_logit_diff:.4f}")

corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_token_indices).item()
print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")

Clean logit diff: 3.5519
Corrupted logit diff: -3.5519


In [28]:
CLEAN_BASELINE = clean_logit_diff
CORRUPTED_BASELINE = corrupted_logit_diff


def ioi_metric(logits, answer_token_indices=answer_token_indices):
    return (get_logit_diff(logits, answer_token_indices) - CORRUPTED_BASELINE) / (
        CLEAN_BASELINE - CORRUPTED_BASELINE
    )


print(f"Clean Baseline is 1: {ioi_metric(clean_logits).item():.4f}")
print(f"Corrupted Baseline is 0: {ioi_metric(corrupted_logits).item():.4f}")

Clean Baseline is 1: 1.0000
Corrupted Baseline is 0: 0.0000


## Patching

In [29]:
Metric = Callable[[TT["batch_and_pos_dims", "d_model"]], float]

In [30]:
for h in model.hook_dict:
    if "mlp" in h:
        print(h)

blocks.0.mlp.hook_pre
blocks.0.mlp.hook_post
blocks.0.hook_mlp_in
blocks.0.hook_mlp_out
blocks.1.mlp.hook_pre
blocks.1.mlp.hook_post
blocks.1.hook_mlp_in
blocks.1.hook_mlp_out
blocks.2.mlp.hook_pre
blocks.2.mlp.hook_post
blocks.2.hook_mlp_in
blocks.2.hook_mlp_out
blocks.3.mlp.hook_pre
blocks.3.mlp.hook_post
blocks.3.hook_mlp_in
blocks.3.hook_mlp_out
blocks.4.mlp.hook_pre
blocks.4.mlp.hook_post
blocks.4.hook_mlp_in
blocks.4.hook_mlp_out
blocks.5.mlp.hook_pre
blocks.5.mlp.hook_post
blocks.5.hook_mlp_in
blocks.5.hook_mlp_out
blocks.6.mlp.hook_pre
blocks.6.mlp.hook_post
blocks.6.hook_mlp_in
blocks.6.hook_mlp_out
blocks.7.mlp.hook_pre
blocks.7.mlp.hook_post
blocks.7.hook_mlp_in
blocks.7.hook_mlp_out
blocks.8.mlp.hook_pre
blocks.8.mlp.hook_post
blocks.8.hook_mlp_in
blocks.8.hook_mlp_out
blocks.9.mlp.hook_pre
blocks.9.mlp.hook_post
blocks.9.hook_mlp_in
blocks.9.hook_mlp_out
blocks.10.mlp.hook_pre
blocks.10.mlp.hook_post
blocks.10.hook_mlp_in
blocks.10.hook_mlp_out
blocks.11.mlp.hook_pre
block

In [31]:
def filter_mlp_hooks(name):
    if "hook_mlp_in" in name or "hook_mlp_out" in name:
        return True
    if "mlp.hook_pre" in name or "mlp.hook_post" in name:
        return True

    return False



def get_cache_fwd_and_bwd(model, tokens, metric):
    model.reset_hooks()
    cache = {}

    def forward_cache_hook(act, hook):
        cache[hook.name] = act.detach()

    model.add_hook(filter_mlp_hooks, forward_cache_hook, "fwd")

    grad_cache = {}

    def backward_cache_hook(act, hook):
        grad_cache[hook.name] = act.detach()

    model.add_hook(filter_mlp_hooks, backward_cache_hook, "bwd")

    value = metric(model(tokens))
    value.backward()
    model.reset_hooks()
    return (
        value.item(),
        ActivationCache(cache, model),
        ActivationCache(grad_cache, model),
    )


clean_value, clean_cache, clean_grad_cache = get_cache_fwd_and_bwd(
    model, clean_tokens, ioi_metric
)
print("Clean Value:", clean_value)
print("Clean Activations Cached:", len(clean_cache))
print("Clean Gradients Cached:", len(clean_grad_cache))
corrupted_value, corrupted_cache, corrupted_grad_cache = get_cache_fwd_and_bwd(
    model, corrupted_tokens, ioi_metric
)
print("Corrupted Value:", corrupted_value)
print("Corrupted Activations Cached:", len(corrupted_cache))
print("Corrupted Gradients Cached:", len(corrupted_grad_cache))

Clean Value: 1.0
Clean Activations Cached: 48
Clean Gradients Cached: 48
Corrupted Value: 0.0
Corrupted Activations Cached: 48
Corrupted Gradients Cached: 48


#### Printing neurons

In [58]:
print(clean_cache['blocks.0.mlp.hook_pre'].shape)
print(clean_cache['blocks.0.hook_mlp_in'].shape)

print(corrupted_cache['blocks.0.mlp.hook_pre'].shape)
print(corrupted_cache['blocks.0.hook_mlp_in'].shape)

print(corrupted_grad_cache['blocks.0.mlp.hook_pre'].shape)
print(corrupted_grad_cache['blocks.0.hook_mlp_in'].shape)

torch.Size([8, 15, 3072])
torch.Size([8, 15, 768])
torch.Size([8, 15, 3072])
torch.Size([8, 15, 768])
torch.Size([8, 15, 3072])
torch.Size([8, 15, 768])


## Attribution patching

In [63]:
def stack_mlp_from_cache(cache, activation_name: Literal["mlp_in", "mlp_out", "pre", "post"]):
    stacked_mlp_vectors = torch.stack(
        [cache[activation_name, l] for l in range(model.cfg.n_layers)], dim=0
    )

    return stacked_mlp_vectors


stack_mlp_from_cache(clean_cache, "mlp_in")
2

2

In [78]:
def attr_patch_mlp(clean_cache, corrupted_cache, grad_cache, activation_name: Literal["mlp_in", "mlp_out", "pre", "post"]):
    labels = [str(i) for i in range(model.cfg.n_layers)]

    clean_act = stack_mlp_from_cache(clean_cache, activation_name)
    corrupted_act = stack_mlp_from_cache(corrupted_cache, activation_name)
    grad_vector = stack_mlp_from_cache(grad_cache, activation_name)

    attribution_vector = einops.reduce(
        grad_vector * (clean_act - corrupted_act),
        "layer batch pos dim -> layer dim",
        "sum"
    )

    #attribution_vector = einops.rearrange(attribution_vector, "layer dim -> dim layer")

    return attribution_vector, labels

for hook_name, descriptive_name in [("mlp_in", "MLP Input"),("mlp_out", "MLP output"),("pre", "MLP neurons after first linear"),("post", "MLP neurons after activation")]:
    attr, names = attr_patch_mlp(clean_cache, corrupted_cache, corrupted_grad_cache, hook_name)

    imshow(
        attr,
        y=names,
        yaxis="Layer No.",
        xaxis="Neuron No.",
        title=f"Attribution of {descriptive_name}, summed over position"
    )